<a href="https://colab.research.google.com/github/Alvaroguijarro97/ML_Group_Project/blob/main/Sampling_and_Threshold_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Undersampling and Oversampling

In [ ]:
# Implementation of Roughly Balanced Bagging for binary problem.

# using a RandomUnderSampler or RandomOverSampler will create exactly the desired number of samples
# However, it does not follow the statistical spirit wanted in the bagging framework.

# not working in the moment

from collections import Counter
from imblearn import FunctionSampler
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.model_selection import cross_validate

def roughly_balanced_bagging(X, y, replace=False):
    
    # find the minority and majority classes
    class_counts = Counter(y)
    majority_class = max(class_counts, key=class_counts.get)
    minority_class = min(class_counts, key=class_counts.get)

    # compute the number of sample to draw from the majority class using
    # a negative binomial distribution
    n_minority_class = class_counts[minority_class]
    n_majority_resampled = np.random.negative_binomial(n=n_minority_class, p=0.5)

    # draw randomly with or without replacement
    majority_indices = np.random.choice(
        np.flatnonzero(y == majority_class),
        size=n_majority_resampled,
        replace=replace,
    )
    minority_indices = np.random.choice(
        np.flatnonzero(y == minority_class),
        size=n_minority_class,
        replace=replace,
    )
    indices = np.hstack([majority_indices, minority_indices])

    return X[indices], y[indices]


# Roughly Balanced Bagging
rbb = BalancedBaggingClassifier(
    sampler=FunctionSampler(func=roughly_balanced_bagging, kw_args={"replace": True})
)
cv_results = cross_validate(rbb, X, y, scoring="balanced_accuracy")

print(f"{cv_results['test_score'].mean():.3f} +/- {cv_results['test_score'].std():.3f}")

# Best Threshold Evaluation Metric

In [ ]:
# This is only an example by now
## Another option would be to change our Evaluation Metric in accordance to the imbalance
# function to get the best threshold for maximizing F1 score for binary predictions

def bestThresshold(y_true,y_pred):
    best_thresh = None
    best_score = 0
    for thresh in np.arange(0.1, 0.501, 0.01):
        score = f1_score(y_true, np.array(y_pred)>thresh)
        if score > best_score:
            best_thresh = thresh
            best_score = score
    return best_score , best_thresh